In [281]:
import numpy as np
import pandas as pd
from datetime import datetime,time
import matplotlib.pyplot as plt
from itertools import product

In [234]:
date_parser=lambda x : datetime.strptime(x,'%Y-%m-%d %H:%M:%S')

In [235]:
appts_data=pd.read_csv('appointment_requests.csv',parse_dates=['session_time'],date_parser=date_parser)

In [239]:
appts_data.head()

,session_time,id,status,appointment_type,session_gender_single,session_type_single,session_gender_double_1,session_gender_double_2,session_type_double_1,session_type_double_2,filled
0,2017-01,2570736,cancelled,couples,NaN,NaN,female,female,swedish,swedish,0
1,2017-01,2570976,cancelled,couples,NaN,NaN,either,either,swedish,swedish,0
2,2017-01,2571172,cancelled,single,female,deep,NaN,NaN,NaN,NaN,0
3,2017-01,2588747,cancelled,single,male,deep,NaN,NaN,NaN,NaN,0
4,2017-01,2588886,cancelled,single,female,deep,NaN,NaN,NaN,NaN,0


In [237]:
appts_data.session_time=time_parser_to_yr_mo(appts_data.session_time)

In [322]:
def time_parser_to_yr_mo(time_data):
    time_parser=lambda x : datetime.strftime(x,'%Y-%m')
    return time_data.apply(time_parser)

def fill_rate_by_month(df):
    d_range=pd.date_range(end=datetime.now(),freq='MS',periods=4,closed='left',normalize=True)-1
    df['filled']=df.status.apply(lambda x : 1 if x=='completed' else 0)
    #print(type(df))
    print(df[df.session_time.isin(d_range.strftime('%Y-%m'))]['filled'].groupby([df.session_time]).mean())
    #df[df.session_time.isin(d_range.strftime('%Y-%m'))]['filled'].groupby([df.session_time]).mean().plot(kind='bar')
    #plt.show()
    
def fill_rate_by_gender(df):
    df1=df.copy()
    fill_and_demand=dict()
    #demand=(appts_data[appts_data.session_time==datetime.now().strftime('%Y-%m')]['session_gender_single'].value_counts()+appts_data[appts_data.session_time==datetime.now().strftime('%Y-%m')]['session_gender_double_1'].value_counts()+appts_data[appts_data.session_time==datetime.now().strftime('%Y-%m')]['session_gender_double_2'].value_counts())*pd.Series(data=[1,1,0.5],index=['female','male','either'])
    df2=df1[df1.session_time==datetime.now().strftime('%Y-%m')].copy()
    demand=pd.Series(data=[0,0,0],index=['female','male','either'])
    fill=pd.Series(data=[0,0,0],index=['female','male','either'])
    for col in ['session_gender_single','session_gender_double_1','session_gender_double_2']:
        demand+=df2[col].value_counts()
        fill+=df2[df2.status=='completed'][col].value_counts()
    multiplier=pd.Series(data=[1,1,0.5],index=['female','male','either'])
    demand*=multiplier
    fill*=multiplier
    for r in list(product(['female','male'],['demand','fill','fill_rate'])):
        if not r[0] in fill_and_demand:
            fill_and_demand[r[0]]=dict()
        if r[1]=='demand':
            fill_and_demand[r[0]][r[1]]=demand.ix[r[0]]+demand.ix['either']
        elif r[1]=='fill':
            fill_and_demand[r[0]][r[1]]=fill.ix[r[0]]+demand.ix['either']
        else:
            fill_and_demand[r[0]][r[1]]=(fill.ix[r[0]]+fill.ix['either'])/(demand.ix[r[0]]+demand.ix['either'])
            
    print(fill_and_demand)

In [323]:
fill_rate_by_gender(appts_data)

{'female': {'fill': 226.0, 'demand': 236.0, 'fill_rate': 0.94067796610169496}, 'male': {'fill': 75.0, 'demand': 77.0, 'fill_rate': 0.92207792207792205}}


In [324]:
appts_data.session_time[:1]

0    2017-01
Name: session_time, dtype: object

In [384]:
appts_data1=pd.read_csv('appointment_requests.csv',parse_dates=['session_time'],date_parser=date_parser)

In [385]:
appts_data1.index=appts_data1.session_time

In [386]:
appts_data1.index=appts_data1.index.to_period('D')

In [387]:
appts_data1['filled']=appts_data1.status.apply(lambda x : 1 if x =='completed' else 0 )

In [388]:
d_range=(pd.date_range(end=datetime.now(),freq='MS',periods=4,closed='left',normalize=True)-1).to_period('M')

In [389]:
appts_data1['filled'].resample('M').mean()[d_range]

2017-06    0.953079
2017-07    0.943662
2017-08    0.934473
2017-09    0.916427
Freq: M, Name: filled, dtype: float64

In [376]:
appts_data2=pd.read_csv('appointment_requests.csv',parse_dates=['session_time'],date_parser=date_parser)

In [377]:
df2=appts_data2[(appts_data2.session_time>=datetime(2017,10,1)) ]

In [378]:
df2['session_time']=df2['session_time'].apply(lambda x: x.strftime('%H'))

/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [379]:
male_dict={'male':1,'either':0.5}
female_dict={'female':1,'either':0.5}
df2['male']=df2.session_gender_single.map(male_dict).fillna(0)+df2.session_gender_double_1.map(male_dict).fillna(0)+df2.session_gender_double_2.map(male_dict).fillna(0)
df2['female']=df2.session_gender_single.map(female_dict).fillna(0)+df2.session_gender_double_1.map(female_dict).fillna(0)+df2.session_gender_double_2.map(female_dict).fillna(0)

/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [381]:
df2.groupby('session_time')[['male','female']].sum()

,male,female
session_time,,
00,7.5,29.5
01,9.0,21.0
02,4.5,19.5
03,4.5,17.5
12,0.0,3.0
13,2.0,5.0
14,3.0,20.0
15,2.0,12.0
16,3.0,10.0


In [406]:
appts_data1['filled'].index=appts_data1['filled'].index.to_timestamp().tz_localize('UTC')

In [410]:
appts_data1['filled'].index.tz_convert('America/New_York')

DatetimeIndex(['2016-12-31', '2016-12-31', '2016-12-31', '2017-01-04',
               '2017-01-04', '2017-01-04', '2017-01-07', '2017-01-06',
               '2017-01-06', '2017-01-06',
               ...
               '2017-10-22', '2017-10-22', '2017-10-22', '2017-10-22',
               '2017-10-22', '2017-10-22', '2017-10-22', '2017-10-22',
               '2017-10-23', '2017-10-23'],
              dtype='datetime64[ns, America/New_York]', name='session_time', length=3322, freq=None)

Help on function pivot_table in module pandas.tools.pivot:

pivot_table(data, values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, dropna=True, margins_name='All')
    Create a spreadsheet-style pivot table as a DataFrame. The levels in the
    pivot table will be stored in MultiIndex objects (hierarchical indexes) on
    the index and columns of the result DataFrame
    
    Parameters
    ----------
    data : DataFrame
    values : column to aggregate, optional
    index : column, Grouper, array, or list of the previous
        If an array is passed, it must be the same length as the data. The list
        can contain any of the other types (except list).
        Keys to group by on the pivot table index.  If an array is passed, it
        is being used as the same manner as column values.
    columns : column, Grouper, array, or list of the previous
        If an array is passed, it must be the same length as the data. The list
        can contain 

In [425]:
pd.pivot_table(df2,index=['session_time'],columns=['status'],values=['male','female'],aggfunc=np.sum,fill_value='-',margins=True)

male                    female                 
status       cancelled completed   All cancelled completed    All
session_time                                                     
00                   1       6.5   7.5         3      26.5   29.5
01                   0       9.0   9.0         1      20.0   21.0
02                 2.5       2.0   4.5       3.5      16.0   19.5
03                   1       3.5   4.5         5      12.5   17.5
12                   -       0.0   0.0         -       3.0    3.0
13                   -       2.0   2.0         -       5.0    5.0
14                   -       3.0   3.0         -      20.0   20.0
15                 0.5       1.5   2.0       0.5      11.5   12.0
16                 0.5       2.5   3.0       0.5       9.5   10.0
17                   -       3.0   3.0         -      12.0   12.0
18                   -       3.5   3.5         -      10.5   10.5
19                   -       8.0   8.0         -      10.0   10.0
20                   -       6.0   6.0         -      20.0   20.0
21                   -       3.5   3.5         -      11.5   11.5
22                 0.5       8.0   8.5       0.5      15.0   15.5
23                   -       9.0   9.0         -      19.0   19.0
All                  6      71.0  77.0        14     222.0  236.0

In [452]:
a=pd.crosstab(df2.appointment_type,df2.status)

In [453]:
b=pd.concat([a,(a.completed/(a.completed+a.cancelled)).rename('fill_rate')],axis=1)

In [456]:
b.fill_rate=b.fill_rate.apply(lambda x : '{0:.0f}%'.format(x*100))

In [457]:
b

,cancelled,completed,fill_rate
appointment_type,,,
back_to_back,0,5,100%
couples,2,23,92%
single,16,242,94%


In [503]:
date_parser2=lambda x : datetime.strptime(x,'%Y-%m-%d %H:%M:%S')
ap_data=pd.read_csv('appointments.csv',date_parser=date_parser2,parse_dates=['created_at','session_time'])

In [504]:
ap_data.created_at=ap_data.created_at.apply(lambda x : x.strftime('%Y-%m'))
ap_data.session_time=ap_data.session_time.apply(lambda x : x.strftime('%Y-%m'))

In [505]:
ap_data.head()

,referral,city,id,soothe_rating,suspended,created_at,session_time
0,NaN,Atlanta,69222,0,1,2015-10,2017-05
1,NaN,Atlanta,69222,0,1,2015-10,2017-07
2,NaN,Atlanta,69222,0,1,2015-10,2017-05
3,Lovelace Linares,Atlanta,79388,4,0,2015-11,2017-01
4,Lovelace Linares,Atlanta,79388,4,0,2015-11,2017-01


In [500]:
ap_data.drop_duplicates(subset=['id'],inplace=True)

In [507]:
pd.pivot_table(ap_data[(ap_data.soothe_rating>0) & (ap_data.suspended==0) & (ap_data.created_at>='2017-07')],index=['referral'],columns=['created_at'],values=['id'],aggfunc=len,fill_value=0)

id                
created_at       2017-07 2017-08 2017-10
referral                                
Clea Etheridge         0       0       2
Lovelace Linares     124      31       0

In [506]:
ap_data[(ap_data.soothe_rating>0) & (ap_data.suspended==0) & (ap_data.created_at>='2017-07')]

,referral,city,id,soothe_rating,suspended,created_at,session_time
3202,Lovelace Linares,Atlanta,712125,4,0,2017-07,2017-09
3203,Lovelace Linares,Atlanta,712125,4,0,2017-07,2017-07
3204,Lovelace Linares,Atlanta,720200,4,0,2017-07,2017-07
3205,Lovelace Linares,Atlanta,720200,4,0,2017-07,2017-07
3206,Lovelace Linares,Atlanta,720200,4,0,2017-07,2017-07
3207,Lovelace Linares,Atlanta,720200,4,0,2017-07,2017-07
3208,Lovelace Linares,Atlanta,720200,4,0,2017-07,2017-07
3209,Lovelace Linares,Atlanta,720200,4,0,2017-07,2017-08
3210,Lovelace Linares,Atlanta,720200,4,0,2017-07,2017-08
3211,Lovelace Linares,Atlanta,720200,4,0,2017-07,2017-08
